# Debugging Individual Agents in AI Project Analizer

This notebook lets you run and debug each BeeAI agent in isolation. Each section will:
1. Import the necessary agent class.
2. Prepare a minimal environment (e.g., a test ZIP or test files).
3. Call the agent's `handle()` method with simulated events.
4. Capture and display outputs (logs, memory writes, emitted events).

Make sure you have extracted the project into the folder:
```
/mnt/c/blog/ai-project-analizer
```


In [2]:
# Setup paths and logging
import sys, os, shutil, zipfile, tempfile
from pathlib import Path

# Add project src and root to sys.path
sys.path.append(r"/mnt/c/blog/ai-project-analizer/src")
sys.path.append(r"/mnt/c/blog/ai-project-analizer")

# Ensure logs are printed
import logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s %(levelname)s %(message)s")

# Create a temporary directory for tests
TEST_DIR = Path("/mnt/c/blog/ai-project-analizer/backup/agent_debug_tests")
if TEST_DIR.exists():
    shutil.rmtree(TEST_DIR)
TEST_DIR.mkdir(parents=True, exist_ok=True)

print("TEST_DIR:", TEST_DIR)


TEST_DIR: /mnt/c/blog/ai-project-analizer/backup/agent_debug_tests


## Create a Sample ZIP for Testing

We'll generate a small ZIP file containing a single `foo.txt`.

In [3]:
# Create a simple zip file with one text file
sample_zip_path = TEST_DIR / "test.zip"
with zipfile.ZipFile(sample_zip_path, "w") as zf:
    # Create a text file inside a temp location
    tmp_file = TEST_DIR / "foo.txt"
    tmp_file.write_text("Hello, world!")
    zf.write(tmp_file, arcname="foo.txt")
print("Created zip at:", sample_zip_path)

Created zip at: /mnt/c/blog/ai-project-analizer/backup/agent_debug_tests/test.zip


## Debug `ZipValidatorAgent`

Instantiate `ZipValidatorAgent`, monkey-patch its `emit` to capture events, and run with our test ZIP.

In [4]:
from src.agents.zip_validator_agent import ZipValidatorAgent

# Prepare the agent
zip_agent = ZipValidatorAgent()
emitted = []
# Monkey-patch emit to capture events
zip_agent.emit = lambda event_name, payload: emitted.append((event_name, payload))

# Test with valid ZIP
event = {"type": "NewUpload", "zip_path": str(sample_zip_path)}
zip_agent.handle(event)
print("Emitted events for valid ZIP:", emitted)

# Test with an invalid path
emitted.clear()
event_bad = {"type": "NewUpload", "zip_path": str(TEST_DIR / "nonexistent.zip")}
zip_agent.handle(event_bad)
print("Emitted events for bad ZIP:", emitted)

ModuleNotFoundError: No module named 'beeai_framework.agent'

## Debug `ExtractionAgent`

Use the valid ZIP event and capture `FileDiscovered` and `ExtractionDone`.

In [ ]:
from src.agents.extraction_agent import ExtractionAgent

# Prepare the agent
ext_agent = ExtractionAgent()
emitted_ext = []
ext_agent.emit = lambda event_name, payload: emitted_ext.append((event_name, payload))

# Run extraction on the valid ZIP event
event_zip_valid = {"type": "ZipValid", "zip_path": str(sample_zip_path)}
ext_agent.handle(event_zip_valid)
print("Extraction emitted events:")
for evt in emitted_ext:
    print(evt)

# Check extracted directory
if emitted_ext and emitted_ext[-1][0] == "ExtractionDone":
    base_dir = Path(emitted_ext[-1][1]["base_dir"])
    print("Contents of extracted directory:", list(base_dir.rglob("*")))


## Debug `TreeBuilderAgent`

Simulate `FileDiscovered` events, then `ExtractionDone` with the previously extracted folder.

In [ ]:
from src.agents.tree_builder_agent import TreeBuilderAgent

# Find the base_dir from previous extraction
if emitted_ext and emitted_ext[-1][0] == "ExtractionDone":
    base_dir = Path(emitted_ext[-1][1]["base_dir"])
else:
    raise RuntimeError("Extraction did not complete successfully.")

# Prepare tree builder
tree_agent = TreeBuilderAgent()
# Send FileDiscovered events (for each file in base_dir)
for file_path in base_dir.rglob("*"):
    if file_path.is_file():
        file_event = {"type": "FileDiscovered", "path": str(file_path)}
        tree_agent.handle(file_event)

# Now fire ExtractionDone
done_event = {"type": "ExtractionDone", "base_dir": str(base_dir)}
tree_agent.handle(done_event)

# The tree text is stored in memory
print("Generated project_tree.txt:")
print(tree_agent.memory.get("project_tree.txt", "<none>"))


## Debug `FileTriageAgent`

Queue discovered files, flush on `ExtractionDone`, and capture `FileForAnalysis` and `TriageComplete`.

In [ ]:
from src.agents.file_triage_agent import FileTriageAgent

# Prepare file triage agent
triage_agent = FileTriageAgent()
emitted_triage = []
triage_agent.emit = lambda event_name, payload: emitted_triage.append((event_name, payload))

# Send FileDiscovered events for all files under base_dir
for file_path in base_dir.rglob("*"):
    if file_path.is_file():
        file_event = {"type": "FileDiscovered", "path": str(file_path)}
        triage_agent.handle(file_event)

# Now send ExtractionDone
done_event = {"type": "ExtractionDone", "base_dir": str(base_dir)}
triage_agent.handle(done_event)

print("Triage emitted events:")
for evt in emitted_triage:
    print(evt)


## Debug `FileAnalysisAgent`

Analyze each file from `FileForAnalysis` events and finalize on `TriageComplete`. Display `file_summaries.json`.

In [ ]:
from src.agents.file_analysis_agent import FileAnalysisAgent
import json

# Prepare file analysis agent
analysis_agent = FileAnalysisAgent()
emitted_analysis = []
analysis_agent.emit = lambda event_name, payload: emitted_analysis.append((event_name, payload))

# Simulate FileForAnalysis events from triage
file_for_analysis_events = [evt for evt in emitted_triage if evt[0] == "FileForAnalysis"]
for _, payload in file_for_analysis_events:
    evt = {"type": "FileForAnalysis", "path": payload["path"], "score": payload.get("score", 0)}
    analysis_agent.handle(evt)

# Now fire TriageComplete
tc_event = {"type": "TriageComplete"}
analysis_agent.handle(tc_event)

print("FileAnalysis emitted events:")
for evt in emitted_analysis:
    print(evt)

# Check memory for file_summaries.json
print("Memory file_summaries.json:")
print(analysis_agent.memory.get("file_summaries.json", "<none>"))


## Debug `SummarySynthesizerAgent`

Feed `TreeBuilt` and `FileAnalysed` / `AnalysisComplete` events, then inspect `project_summary.txt`.

In [ ]:
from src.agents.summary_synthesizer_agent import SummarySynthesizerAgent

# Prepare summary synthesizer agent
summary_agent = SummarySynthesizerAgent()
emitted_summary = []
summary_agent.emit = lambda event_name, payload: emitted_summary.append((event_name, payload))

# Manually populate memory for tree text and file summaries
summary_agent.memory["project_tree.txt"] = tree_agent.memory.get("project_tree.txt", "")
# Use the list of analyses from emitted_analysis if any; else simulate one
analyses = [payload for (name, payload) in emitted_analysis if name == "FileAnalysed"]
for analysis in analyses:
    evt = {"type": "FileAnalysed", **analysis}
    summary_agent.handle(evt)

# Now signal AnalysisComplete
ac_event = {"type": "AnalysisComplete"}
summary_agent.handle(ac_event)

# Also signal TreeBuilt (to ensure it has the tree)
tb_event = {"type": "TreeBuilt", "tree_path": "project_tree.txt"}
summary_agent.handle(tb_event)

print("SummarySynthesizer emitted events:")
for evt in emitted_summary:
    print(evt)

# Check memory for project_summary.txt
print("\nGenerated project_summary.txt:")
print(summary_agent.memory.get("project_summary.txt", "<none>"))